In [1]:
# permutations that require max number of dcj operations  - diameter of an instance 

# find the largest value of n which you can find the diameter 
# n --> diameter (randomly sample x number)
# increase n  

In [2]:
using Random

using NBInclude
@nbinclude("dcj_algo.ipynb")

In [3]:
# Function to get the first n letters of the alphabet
function first_n_letters(n::Int)
    return collect('a':'z')[1:n]
end

# generates genomes/permutations/strings of length n 
# (up to 26 bc alphabet)
function generate_permutation(n::Int)
    letters = first_n_letters(n)
    return join(shuffle(letters))
end

generate_permutation (generic function with 1 method)

In [4]:
function calculate_diameter(n::Int, num_src_permutations_to_generate::Int)
    x = 10  # hyperparam -- tune
    target = generate_permutation(n) 
    max_dist = 0 
    max_dist_permutation = ""

    for i in 1:num_src_permutations_to_generate
        src = generate_permutation(n)

        dist = calculate_distance(src, target)
        if max_dist < dist 
            max_dist = dist
            max_dist_permutation = src 
        end 
    end 

    return max_dist, max_dist_permutation
end 

function calculate_diameters(n::Int) 
    diameters = Vector{Tuple{Int, String}}()

    for i in 1:n
        # for x in 1:
        x = 1
        diam, max_dist_permutation = calculate_diameter(i, x) 

        push!(diameters, (diam, max_dist_permutation))  # i == idx
    end 

    return diameters
end 

n = 10 
calculate_diameters(n)

SRC (a:h,a:t) --> TARGET (a:h,a:t)

*************
number of operations = 0
--------------------------------------------------------------------------------------
SRC (b:h,a:t)(a:h,b:t) --> TARGET (b:h,a:t)(a:h,b:t)

*************
number of operations = 0
--------------------------------------------------------------------------------------
SRC (b:h,c:t)(c:h,a:t)(a:h,b:t) --> TARGET (a:h,b:t)(b:h,c:t)(c:h,a:t)

*************
number of operations = 0
--------------------------------------------------------------------------------------
SRC (b:h,d:t)(d:h,a:t)(a:h,c:t)(c:h,b:t) --> TARGET (a:h,c:t)(c:h,d:t)(d:h,b:t)(b:h,a:t)

*************
DCJ Operation :: (c:h,b:t)(b:h,d:t) --> (c:h,d:t)(b:t,b:h)
DCJ Operation :: (d:h,a:t)(b:t,b:h) --> (d:h,b:t)(a:t,b:h)
number of operations = 2
--------------------------------------------------------------------------------------
SRC (a:h,b:t)(b:h,c:t)(c:h,e:t)(e:h,d:t)(d:h,a:t) --> TARGET (b:h,e:t)(e:h,c:t)(c:h,a:t)(a:h,d:t)(d:h,b:t)

*************
DCJ 

10-element Vector{Tuple{Int64, String}}:
 (0, "")
 (0, "")
 (0, "")
 (2, "bdac")
 (4, "abced")
 (4, "cdeabf")
 (4, "cfbgdea")
 (6, "afcedhbg")
 (6, "bhifeacdg")
 (6, "cbiehfdgja")